In [2]:
# import itertools as it
# from multiprocessing import Pool, cpu_count
# from collections import defaultdict
import re
import numpy as np
from functools import reduce
from operator import mul
with open('input.txt', 'r') as file:
    input = file.read()

test_input = """\
p=0,4 v=3,-3
p=6,3 v=-1,-3
p=10,3 v=-1,2
p=2,0 v=2,-1
p=0,0 v=1,3
p=3,0 v=-2,-2
p=7,6 v=-1,-3
p=3,0 v=-1,-2
p=9,3 v=2,3
p=7,3 v=-1,2
p=2,4 v=2,-3
p=9,5 v=-3,-3
"""


In [19]:
np.set_printoptions(threshold=np.inf,linewidth=205)
print(np.get_printoptions())
class RobotSearch():
    def __init__(self,txt,w,h,iterations):
        pattern = r"p=(\d+),(\d+) v=(-?\d+),(-?\d+)"
        self.num_robots = len(txt.splitlines())
        self.infos = [[(int(px),int(py)),(int(vx),int(vy))] for px,py,vx,vy in re.findall(pattern,txt)]
        self.w=w
        self.h=h
        self.np_rep = []
        self.convert_to_np()
        max_safety = 0
        for i in range(iterations):
            self.teleport()
            # safety = self.calc_safety()
            # if safety > max_safety:
            #     max_safety = safety
            #     print("safety: ",safety)
            self.convert_to_np()

            if (self.cluster_heurstics()):
                self.print_as_np()
                print("above: iter=",i)
                print("suspicious cluster!")


    def teleport(self):
        for info in self.infos:
            newX,newY = (info[0][0] + info[1][0])%self.w,(info[0][1] + info[1][1])%self.h
            info[0] = (newX,newY)
    
    def convert_to_np(self):
        mat = [[0 for x in range(self.w)] for y in range(self.h)]
        for (px,py),(vx,vy) in self.infos:
            # print(px,'',py)
            if mat[py][px] == 0:
                mat[py][px] = 1
            else:
                mat[py][px] = int(mat[py][px])+1
        arr_np = np.array(mat)
        self.np_rep = arr_np
        # print(arr_np)
        # Replace 0 with " " or "."

    def print_as_np(self):
        printable = np.where(self.np_rep == 0, ".", self.np_rep)  # Replace with "."; use " " instead if desired
        
        # Convert to string for display
        printable_str = "\n".join("".join(str(cell) for cell in row) for row in printable)
        print(printable_str) 
        
    def calc_safety(self):
        # print(self.np_rep)
        return reduce(mul,[np.sum(q) for q in self.get_quads()])
    
    def get_quads(self):
        halfW,halfH= self.w//2,self.h//2
        q1 = self.np_rep[:halfH,:halfW]
        # print(q1)
        q2 = self.np_rep[halfH+1:,:halfW]
        # print(q2)
        q3 = self.np_rep[:halfH,halfW+1:]
        # print(q3)
        q4 = self.np_rep[halfH+1:,halfW+1:]
        # print(q4)
        return [q1,q2,q3,q4]

    def cluster_heurstics(self):
        quads = self.get_quads()
        half_num = self.num_robots/3
        for q in quads:
            if np.sum(q) >half_num:
                return True
        return False


res = RobotSearch(input,101,103,10000)
print(res.calc_safety())


{'edgeitems': 3, 'threshold': inf, 'floatmode': 'maxprec', 'precision': 8, 'suppress': False, 'linewidth': 205, 'nanstr': 'nan', 'infstr': 'inf', 'sign': '-', 'formatter': None, 'legacy': False}
............................................1.......1......................................1.........
..................................................................................................1..
...................................................................................1.................
..........................................................................1...................1......
....................................................................................1.......1........
......................................................................1..............................
.....................................................................................................
.....................1................................................................1.1..

### Part 1
A lot of matrix manipulation! I got to practice numpy, especially with the quadrant division, which I spent the longest in. Took me a while to also print the visualization correctly, but it was worth it for part 2

### Part 2
At first I was completely at a loss for what to do, so I just manually printed out each loop and checked. It was taking a good 3+ min before vsc crashed so that wasn't efficient enough at all. I got a hint by looking at reddit for what is the tree I'm looking for, and then tried to re-use what I had from part 1 for a heuristic function for detecting suspicious clusters. Most of the computation was in the printing so as soon as I wasn't manually printing out every single loop the time got cut down by way more. And then it's just sorting through the suspicious clusters for the tree!

In retrospect it seems obvious why there wasn't an example answer but it was a bit of a surprise I guess!

Here's me trying to see if there's any trees in the example set. (There wasn't. CLEARLY the example robots aren't by the elves!)

In [22]:
res = RobotSearch(test_input,11,7,10000)
print(res.calc_safety())

...1...1..1
.1111......
.11.....1..
...........
......1....
...........
1..........
above: iter= 5
suspicious cluster!
.........1.
.......1.1.
...........
..1........
.......1..1
1.....1....
1.12.......
above: iter= 10
suspicious cluster!
..1...2....
...1.11.1..
.1...1....1
...........
..1........
...........
....1......
above: iter= 12
suspicious cluster!
..1........
.1........1
........1..
...11......
...........
...1..2...1
......1...1
above: iter= 14
suspicious cluster!
...........
1.1.....1..
.1.11.11...
..1........
.1.1.......
..........1
...........
above: iter= 16
suspicious cluster!
...........
........1..
.......1.1.
..1....1...
...1..11.1.
..........1
..1.......1
above: iter= 18
suspicious cluster!
.....2...1.
.1..1....2.
.....1.1.1.
...........
.........1.
...........
........1..
above: iter= 19
suspicious cluster!
...........
....1......
..1.......1
.1........1
......2.1.1
...1.......
...1..1....
above: iter= 25
suspicious cluster!
.1..1...1..
.11..2.....
..1.1....1.
.....